In [14]:
import pandas as pd
import pickle
from tqdm import tqdm
import warnings
import gensim
from gensim import corpora, models
from gensim.corpora import Dictionary
warnings.filterwarnings('ignore')
from gensim.models import CoherenceModel #coherence 모델 라이브러리
import matplotlib.pyplot as plt #그래프 그리기
import numpy as np
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, dendrogram

In [15]:
# 클러스터링 파일 가져오기
with open('DX_Actor.pkl', 'rb') as f:
    df = pickle.load(f)

In [16]:
df_cluster0 = df[df['cluster'] == 1]
df_cluster0.head()
len(df_cluster0)

10530

In [17]:
# 단어 사전 만들기
all_documents = list(df_cluster0['tagged_content'])
dictionary = Dictionary(all_documents)
dictionary.token2id

{'가능하다': 0,
 '강등': 1,
 '개': 2,
 '거치': 3,
 '결과': 4,
 '결승전': 5,
 '결정': 6,
 '경기': 7,
 '계정': 8,
 '골': 9,
 '공식': 10,
 '관련': 11,
 '기록': 12,
 '기사': 13,
 '기장': 14,
 '남성': 15,
 '남양주': 16,
 '남지': 17,
 '내': 18,
 '높다': 19,
 '높이': 20,
 '뉴스': 21,
 '당진': 22,
 '대부분': 23,
 '대신': 24,
 '대해': 25,
 '되어다': 26,
 '득실': 27,
 '득점': 28,
 '들다': 29,
 '떠나다': 30,
 '라운드': 31,
 '리그': 32,
 '리드': 33,
 '링크': 34,
 '마무리': 35,
 '만회': 36,
 '머리': 37,
 '무승부': 38,
 '문경': 39,
 '받다': 40,
 '번역': 41,
 '변경': 42,
 '별': 43,
 '보다': 44,
 '보도': 45,
 '보상': 46,
 '불확실하다': 47,
 '브라우저': 48,
 '빠르다': 49,
 '빠지다': 50,
 '사이트': 51,
 '상대로': 52,
 '서울': 53,
 '성명': 54,
 '세종': 55,
 '수': 56,
 '수정': 57,
 '쉬다': 58,
 '승격': 59,
 '승리': 60,
 '시': 61,
 '시민': 62,
 '시작': 63,
 '시즌': 64,
 '시흥': 65,
 '않다': 66,
 '양': 67,
 '얻다': 68,
 '없다': 69,
 '여성': 70,
 '여자': 71,
 '연패': 72,
 '열리다': 73,
 '영어': 74,
 '예선': 75,
 '울산': 76,
 '웹': 77,
 '위': 78,
 '윤균상': 79,
 '이름': 80,
 '인스타그램': 81,
 '일부': 82,
 '자동': 83,
 '자존심': 84,
 '자체': 85,
 '잡지': 86,
 '전': 87,
 '전반': 88,
 '전체': 89,
 '정기'

In [18]:
corpus = []

for doc in all_documents :
    corpus.append(dictionary.doc2bow(doc)) 

In [19]:
# 3개의 토픽을 갖도록 LDA Modeling

topic_num = 3

ldamodel = gensim.models.ldamodel.LdaModel(corpus,
                                           num_topics=topic_num,
                                           id2word=dictionary,
                                           passes=20,
                                           iterations=50,
                                           random_state=42
                                          )

In [ ]:
# 결과 저장 리스트
c_score = [] # Coherence
p_score = [] # Perplexity
# 토픽 개수 범위 (2 ~ 9)
start_topic = 2
end_topic = 10

for i in range(start_topic, end_topic):
    # 1. LDA 모델 생성
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word=dictionary, random_state=42)

    # 2. Coherence Score 계산
    coherencemodel = CoherenceModel(model=ldamodel, texts=df['tagged_content'], dictionary=dictionary, coherence='c_v')
    c_score.append(coherencemodel.get_coherence())

    # 3. Perplexity Score 계산
    p_score.append(ldamodel.log_perplexity(corpus))

    print(f"Topic {i} complete.")


# --- 시각화 (Coherence / Perplexity / Silhouette / Elbow) ---
x_range = range(start_topic, end_topic)

plt.figure(figsize=(24, 5))

# 1. Coherence (높을수록 좋음)
plt.subplot(1, 4, 1)
plt.plot(x_range, c_score, 'o-')
plt.title('Coherence Score')
plt.xlabel('Num Topics')
plt.ylabel('Coherence')
plt.grid(True)

# 2. Perplexity (낮을수록 좋음)
plt.subplot(1, 4, 2)
plt.plot(x_range, p_score, 'o-')
plt.title('Perplexity Score')
plt.xlabel('Num Topics')
plt.ylabel('Perplexity')
plt.grid(True)

plt.tight_layout()
plt.show()

Topic 2 complete.


In [57]:
topic_num = 5

ldamodel = gensim.models.ldamodel.LdaModel(corpus,
                                           num_topics=topic_num,
                                           id2word=dictionary,
                                           passes=20,
                                           iterations=50,
                                           random_state=42
                                          )

In [58]:
ldamodel.get_document_topics(corpus)[1]

[(1, 0.04287682), (2, 0.28410965), (3, 0.2576317), (4, 0.41221714)]

In [59]:
action_align = []

for doc in tqdm(ldamodel.get_document_topics(corpus)) :
    label = []
    value = []
    for score in doc :
        label.append(score[0])
        value.append(score[1])
    max_index = np.argmax(value)
    action_n = label[max_index]
    action_align.append(action_n)

100%|███████████████████████████████████████| 20254/20254 [00:03<00:00, 6552.74it/s]


In [60]:
action_align[:20]

[0, 4, 2, 1, 2, 3, 3, 3, 2, 3, 2, 2, 3, 3, 2, 4, 3, 4, 2, 2]

In [61]:
df_cluster0['action_cluster'] = action_align
df_cluster0

,keyword,title,content,date,href,tagged_content,vector,cluster,action_cluster
0,OTT,오늘 하루는 어땠어?,오늘은요 휴가 일차예요 일은 온전히 나만을 위한 시간을. 나머지 일은 먹는거에 진심...,2025.08.11,https://cafe.naver.com/f-e/cafes/28962370/arti...,"[오늘, 휴가, 일차, 오다, 위, 나머지, 먹다, 진심, 찌다, 친, 먹방, 투어...","[0.22175895, -0.12508897, -0.40555868, -0.3427...",0,0
1,OTT,오늘의 마지막 일정~~,저는 영화 보는 걸 너무 좋아해서 예전에는 일주일에 편씩 보러 갔는데 요즘은 넷플릭...,2025.06.01,https://cafe.naver.com/f-e/cafes/28962370/arti...,"[영화, 보다, 걸, 좋아하다, 예전, 일주일, 편, 보다, 가다, 요즘, 넷플릭스...","[0.33253494, -0.19450366, -0.021705324, -0.259...",0,4
2,OTT,독수리 오형제를 부탁해 정보 줄거리 출연진 다시보기,KBS TV의 주말드라마 독수리 오형제를 부탁해 는 년 월 일부터 방영되었습니다. ...,2025.02.02,https://cafe.naver.com/f-e/cafes/28962370/arti...,"[주말, 드라마, 독수리, 형제, 부탁, 늘다, 방영, 되어다, 드라마, 다리미, ...","[0.34293327, -0.74443346, 0.20365337, -0.64919...",0,2
3,OTT,중증외상센터 | 드라마 정보 | 줄거리 | 등장인물 | 결말 | 시즌2 | 원작 웹...,년 월 일. 넷플릭스에서 공개된 메디컬 드라마 중증외상센터 가 폭발적인 인기를 끌고...,2025.01.30,https://cafe.naver.com/f-e/cafes/28962370/arti...,"[넷플릭스, 공개, 메다, 컬, 드라마, 중증, 외상, 센터, 가다, 폭발, 인기,...","[-0.27246693, -0.2074126, -0.12381751, 0.01281...",0,1
4,OTT,다리미 패밀리 정보 시청률 인기비결 다시보기,KBS 에서 미녀와 순정남 후속 드라마로 방영된 다리미 패밀리 보셨나요? 생각보다 ...,2025.01.12,https://cafe.naver.com/f-e/cafes/28962370/arti...,"[미녀, 정남, 후속, 드라마, 방영, 다리미, 패밀리, 보다, 생각, 재미있다, ...","[0.19679599, -0.41983578, 0.50923896, -0.71487...",0,2
...,...,...,...,...,...,...,...,...,...
35210,핟습자기개발통합,결혼 자녀 2억 보내줬다간~/한국 우유니/캠핑~축구 등,결혼하는 자녀 전세금 억 그냥 보태줬다간 전문가의 조언 https news.nave...,2021.06.28,https://cafe.naver.com/f-e/cafes/23676262/arti...,"[결혼, 자녀, 세금, 억, 그냥, 보태, 주다, 전문가, 조언, 한국, 우유, 페...","[-0.085391104, 0.036018275, -0.13712354, -0.07...",0,1
35213,핟습자기개발통합,50플러스 캠퍼스 신청/IT쇼/농막/축구 등,. 토 kbs http schedule.kbs.co.kr KBS 편성표 대한민국 대...,2021.04.10,https://cafe.naver.com/f-e/cafes/23676262/arti...,"[토, 편성표, 대한민국, 대표, 공영, 미디어, 시니어, 토크쇼, 황금, 연못, ...","[-0.3766906, -0.29118583, -0.2752183, -0.68805...",0,3
35214,핟습자기개발통합,50plus센터,Web발신 서대문 플러스센터 에게 모두 드리리 등산교실부터 손바느질 특강까지 꽉꽉 ...,2021.04.08,https://cafe.naver.com/f-e/cafes/23676262/arti...,"[발신, 서대문, 플러스, 센터, 드리다, 등산, 교실, 손바느질, 특강, 채우다,...","[-0.104748785, -0.35709068, -0.15233357, -0.06...",0,3
35215,핟습자기개발통합,시골 공동체마을/알뜰신잡job/드론촬영/문화/숲캉스 등,. 화 교육방송 스리랑카 마술사부부 홋카이도 미술관.건축탐구 집 시골 공동체마을 h...,2021.03.23,https://cafe.naver.com/f-e/cafes/23676262/arti...,"[화, 교육방송, 스리랑카, 마술사, 부부, 홋카이도, 미술관, 건축, 탐구, 집,...","[0.14320256, -0.16392611, -0.364521, -0.221858...",0,3


In [62]:
df_cluster0['action_cluster'].value_counts()

action_cluster
0    6499
4    5736
2    5148
3    1787
1    1084
Name: count, dtype: int64

# ◆ 3. LDA 시각화 (LDAvis)


In [63]:
# !pip install pyLDAvis

In [64]:
#경고무시
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

#LDA 시각화 라이브러리
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [65]:
prepared_data = gensimvis.prepare(ldamodel, corpus, dictionary)

pyLDAvis.save_html(prepared_data, 'ldavis_cluster0_0925.html')

pyLDAvis.display(prepared_data)

# ◆ 4. LDA 분석을 위한 후작업
* LDAvis와 LDA모델의 토픽 넘버 매칭
* LDAvis 결과와 LDA모델의 결과를 보고 직접 매칭해야한다.

In [55]:
topics = ldamodel.show_topics(num_topics=-1, formatted=True)

for topic_id, topic_desc in topics :
    print(f"Topic ID : {topic_id}, Description : {topic_desc}")

Topic ID : 0, Description : 0.014*"먹다" + 0.013*"가다" + 0.011*"오다" + 0.011*"보다" + 0.011*"오늘" + 0.009*"좋다" + 0.007*"아침" + 0.006*"곳" + 0.006*"들다" + 0.006*"집"
Topic ID : 1, Description : 0.006*"만원" + 0.005*"명" + 0.004*"받다" + 0.004*"코로나" + 0.004*"개" + 0.004*"투자" + 0.004*"원" + 0.003*"만" + 0.003*"수" + 0.003*"위"
Topic ID : 2, Description : 0.018*"보다" + 0.012*"생각" + 0.010*"좋다" + 0.009*"들다" + 0.009*"많다" + 0.009*"않다" + 0.009*"분" + 0.008*"글" + 0.008*"싶다" + 0.007*"없다"
Topic ID : 3, Description : 0.022*"여행" + 0.011*"방송" + 0.008*"분" + 0.008*"홈페이지" + 0.007*"한국" + 0.007*"자막" + 0.007*"세계" + 0.006*"시" + 0.006*"곳" + 0.005*"수"
Topic ID : 4, Description : 0.013*"없다" + 0.012*"보다" + 0.009*"그렇다" + 0.009*"않다" + 0.009*"말" + 0.009*"내" + 0.008*"아니다" + 0.008*"가다" + 0.008*"사람" + 0.007*"집"


In [ ]:
0 -> 3
1 -> 4
2 -> 2
3 -> 5
4 -> 1

In [66]:
new_action_topic =[]

for i in tqdm(df_cluster0.action_cluster) :
    if i == 0 :
        n = 3
    elif i == 1:
        n = 4
    elif i == 2:
        n = 2
    elif i == 3:
        n = 5
    else :
        n = 1
    new_action_topic.append(n)

100%|████████████████████████████████████| 20254/20254 [00:00<00:00, 2035788.86it/s]


In [67]:
df_cluster0.action_cluster = new_action_topic
df_cluster0.action_cluster.value_counts()

action_cluster
3    6499
1    5736
2    5148
5    1787
4    1084
Name: count, dtype: int64

In [68]:
df_cluster0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20254 entries, 0 to 35221
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   keyword         20254 non-null  object
 1   title           20254 non-null  object
 2   content         20254 non-null  object
 3   date            20254 non-null  object
 4   href            20254 non-null  object
 5   tagged_content  20254 non-null  object
 6   vector          20254 non-null  object
 7   cluster         20254 non-null  int64 
 8   action_cluster  20254 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 1.5+ MB


In [69]:
df_cluster0 = df_cluster0[['content', 'tagged_content','cluster','action_cluster']]
df_cluster0

,content,tagged_content,cluster,action_cluster
0,오늘은요 휴가 일차예요 일은 온전히 나만을 위한 시간을. 나머지 일은 먹는거에 진심...,"[오늘, 휴가, 일차, 오다, 위, 나머지, 먹다, 진심, 찌다, 친, 먹방, 투어...",0,3
1,저는 영화 보는 걸 너무 좋아해서 예전에는 일주일에 편씩 보러 갔는데 요즘은 넷플릭...,"[영화, 보다, 걸, 좋아하다, 예전, 일주일, 편, 보다, 가다, 요즘, 넷플릭스...",0,1
2,KBS TV의 주말드라마 독수리 오형제를 부탁해 는 년 월 일부터 방영되었습니다. ...,"[주말, 드라마, 독수리, 형제, 부탁, 늘다, 방영, 되어다, 드라마, 다리미, ...",0,2
3,년 월 일. 넷플릭스에서 공개된 메디컬 드라마 중증외상센터 가 폭발적인 인기를 끌고...,"[넷플릭스, 공개, 메다, 컬, 드라마, 중증, 외상, 센터, 가다, 폭발, 인기,...",0,4
4,KBS 에서 미녀와 순정남 후속 드라마로 방영된 다리미 패밀리 보셨나요? 생각보다 ...,"[미녀, 정남, 후속, 드라마, 방영, 다리미, 패밀리, 보다, 생각, 재미있다, ...",0,2
...,...,...,...,...
35210,결혼하는 자녀 전세금 억 그냥 보태줬다간 전문가의 조언 https news.nave...,"[결혼, 자녀, 세금, 억, 그냥, 보태, 주다, 전문가, 조언, 한국, 우유, 페...",0,4
35213,. 토 kbs http schedule.kbs.co.kr KBS 편성표 대한민국 대...,"[토, 편성표, 대한민국, 대표, 공영, 미디어, 시니어, 토크쇼, 황금, 연못, ...",0,5
35214,Web발신 서대문 플러스센터 에게 모두 드리리 등산교실부터 손바느질 특강까지 꽉꽉 ...,"[발신, 서대문, 플러스, 센터, 드리다, 등산, 교실, 손바느질, 특강, 채우다,...",0,5
35215,. 화 교육방송 스리랑카 마술사부부 홋카이도 미술관.건축탐구 집 시골 공동체마을 h...,"[화, 교육방송, 스리랑카, 마술사, 부부, 홋카이도, 미술관, 건축, 탐구, 집,...",0,5


In [72]:
df_cluster0.to_csv('Cluster0_action_5_0925.csv', encoding = 'utf-8-sig')

In [73]:
with open('Cluster0_action_5_0925.pkl', 'wb') as f:
    pickle.dump(df_cluster0, f)